Here we are going to use the data we created in the 'extract-gsc.ipynb' file. 

Thanks Masaki Okazawa for sharing the Twittorial about GSC. I made some nice tweaks as well in the script. But credits to you! 
https://github.com/saksters  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
#from google.colab import drive # << Again, only activate when using just this script
#drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/gsc_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df['rounded_position'] = df['position'].round(0).astype('int64')

In [ ]:
df

In [ ]:
branded = df[~df['query'].str.contains("bcc") == False]
no_pos_1 = branded[branded.rounded_position != 1]
#no_pos_1.to_csv('branded_results_to-improve.csv')

In [ ]:
def brand_non_brand(x):
    if 'bcc' in x:
        return 'Brand'
    else:
        return 'Non-Brand'
df['brand vs. nonbrand'] = df['query'].apply(brand_non_brand)

In [ ]:
df_top_10 = df[df['rounded_position'] <= 10]

In [ ]:
#df

In [ ]:
ctr_pivot = df_top_10.pivot_table(index=('rounded_position'), values=['ctr'], aggfunc={'ctr':"mean"}).sort_values(by=['rounded_position'], ascending=True)
brand_ctr_pivot = df_top_10[df_top_10['brand vs. nonbrand']=='Brand'].pivot_table(index=('rounded_position'), values=['ctr'], aggfunc={'ctr':"mean"}).sort_values(by=['rounded_position'], ascending=True)
non_brand_ctr_pivot = df_top_10[df_top_10['brand vs. nonbrand']=='Non-Brand'].pivot_table(index=('rounded_position'), values=['ctr'], aggfunc={'ctr':"mean"}).sort_values(by=['rounded_position'], ascending=True)
dfs = [ctr_pivot, brand_ctr_pivot, non_brand_ctr_pivot]
#pd.concat(dfs, axis=1)

In [ ]:
df_merged = reduce(lambda left,right:pd.merge(left,right,on=['rounded_position'],how='outer'), dfs)
df_merged.rename(columns={"ctr_x": "ctr_all", "ctr_y": "ctr_brand", "ctr": "ctr_nonbrand"})

In [ ]:
ax = df_merged.head(10).plot(kind='bar')
ax.set_ylabel('site_ctr')
ax.set_title('bcc.nl site ctr curve')
ax.legend(('ctr_all','ctr_brand','ctr_nonbrand'))
ax.plot(figsize=(15, 7))

In [ ]:
brand_ranking_pivot = df[df['brand vs. nonbrand']=='Brand'].pivot_table(index=('rounded_position'), values=['query'], aggfunc={'query':"count"}).sort_values(by=['rounded_position'], ascending=True)
non_brand_ranking_pivot = df[df['brand vs. nonbrand']=='Non-Brand'].pivot_table(index=('rounded_position'), values=['query'], aggfunc={'query':"count"}).sort_values(by=['rounded_position'], ascending=True)
dfs2 = [brand_ranking_pivot, non_brand_ranking_pivot]
df_merged2 = reduce(lambda left,right:pd.merge(left,right,on=['rounded_position'],how='outer'), dfs2)
ax2 = df_merged2.head(10).plot(kind='bar')
ax2.set_ylabel('count of queries')
ax2.set_title('brand and nonbrand ranking distribution')
ax2.legend(('brand_queries','nonbrand_queries'))

In [ ]:
#df.head()

In [ ]:
#df['category']=df['page'].str.extract(r'(?:.*?\/){3}([^\/?#]+)')
df['category']=df['page'].str.extract(r'(?:.*?\/){4}([^\/?#]+)')
df['category']=df['category'].str.replace('-', ' ').str.title()
df.head().style.set_properties(subset=['page'])

In [ ]:
df_brand = df[df['query'].str.contains('bcc')]
df_nonbrand = df[~df['query'].str.contains('bcc')]

In [ ]:
category_pivot = df.pivot_table(index='category', values=['clicks'], aggfunc={'clicks': np.sum})
category_pivot = category_pivot.sort_values(by='clicks', ascending=False)
category_pivot.head(10)

In [ ]:
category_pivot = df.pivot_table(index='category', values=['clicks'], aggfunc={'clicks':np.sum})
brand_category_pivot = df[df['brand vs. nonbrand']=='Brand'].pivot_table(index='category', values=['clicks'], aggfunc={'clicks':np.sum})
non_brand_category_pivot = df[df['brand vs. nonbrand']=='Non-Brand'].pivot_table(index='category', values=['clicks'], aggfunc={'clicks':np.sum})
dfs3 = [category_pivot, brand_category_pivot, non_brand_category_pivot]
df_merged3 = reduce(lambda left,right:pd.merge(left,right,on=['category'],how='outer'), dfs3)
df_merged3.rename(columns={"clicks_x": "clicks_all", "clicks_y": "clicks_brand", "clicks": "clicks_nonbrand"})
df_merged3 = df_merged3.sort_values(by='clicks', ascending=False)

In [ ]:
df_merged3.rename(columns={"clicks_x": "clicks_all", "clicks_y": "clicks_brand", "clicks": "clicks_nonbrand"})
df_merged3.head(10)

In [ ]:
ax_category = df_merged3.head(10).plot(kind='bar')
ax_category.set_ylabel('url clicks')
ax_category.set_title('top 10 clicks by category')
ax_category.legend(('clicks_all','clicks_brand','clicks_nonbrand'))

In [ ]:
keyword_cannibalization_pivot = df.pivot_table(index='query', values=['page'], aggfunc={'page': 'count'})
keyword_cannibalization_pivot = keyword_cannibalization_pivot.sort_values(by='page', ascending=False)
keyword_cannibalization_pivot.head(10)

In [ ]:
most_keywords_pivot = df.pivot_table(index='page', values=['query'], aggfunc={'query': 'count'})
most_keywords_pivot = most_keywords_pivot.sort_values(by='query', ascending=False)
most_keywords_pivot.head(10)